<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="400" alt="cognitiveclass.ai logo"  />
</center>

# Work with missing data, their further interpolation

## Lab 2: Data Wrangling

Estimated time needed: **30** minutes

## Objectives

After completing this lab you will be able to:

*   Handle missing values
*   Correct data format
*   Standardize and normalize data


<h2>Table of Contents</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li><a href="https://#identify_handle_missing_values">1. Identify and handle missing values</a>
        <ul>
            <li><a href="https://#identify_missing_values">Identify missing values</a></li>
            <li><a href="https://#deal_missing_values">Deal with missing values</a></li>
            <li><a href="https://#correct_data_format">Correct data format</a></li>
        </ul>
    </li>
    <li><a href="https://#data_standardization">2. Data standardization</a></li>
    <li><a href="https://#data_normalization">3. Data normalization (centering/scaling)</a></li>
    <li><a href="https://#binning">3. Binning and Indicator variable</a></li>
</ul>

</div>

<hr>


## What is the purpose of data wrangling?


Data wrangling is the process of converting data from the initial format to a format that may be better for analysis.


### Import data</h3>
<p>
Аfter the end of the previous "Lab_1", we created a new dataset called "df_all_1H", which we will use in this laboratory <a href="https://1824251045.rsc.cdn77.org/web/algohouse/data/ETHBUSD_trades_1m.csv">https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0BWOEN/df_all_1H.csv</a>. 
We will be using this dataset throughout this course.
</p>


### Import pandas


If you run the lab locally using Anaconda, you can load the correct library and versions by uncommenting the following:


In [ ]:
#If you run the lab locally using Anaconda, you can load the correct library and versions by uncommenting the following:
#install specific version of libraries used in lab
#! mamba install pandas==1.3.3
#! mamba install numpy=1.21.2
! mamba install scikit-learn -y

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import requests
%matplotlib inline
import matplotlib.pyplot as plt

## Reading the dataset from the URL


First, we assign the URL of the dataset to "filename".


This dataset was hosted on IBM Cloud object. Click <a href="https://1824251045.rsc.cdn77.org/web/algohouse/data/ETHBUSD_trades_1m.csv">HERE</a> for free storage.


In [ ]:
filename = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0BWOEN/df_all_1H.csv"

Use the Pandas method <b>read_csv()</b> to load the data from the web address. Set the parameter  "names" equal to the Python list "headers".


In [ ]:
df = pd.read_csv(filename, index_col=0)
df.index = pd.to_datetime(df.index)

Use the method <b>head()</b> to display the first five rows of the dataframe.


In [ ]:
# To see what the data set looks like, we'll use the head() method.
df.head(40)

Our data now is clean but let's spoil them to show how we can deal with missing values and incorrect data

<div>Let's define <b>spoil_df</b> function which produces incorrect data of 3 types:</div>

<ol>
    <li>Text data (we work with numbers)</li>
    <li>Negative values (we work only with positive values)</li>
    <li>Missing values (NaN)</li>
</ol>

In [ ]:
def spoil_df(df: pd.DataFrame, cols: list = ["size BID", "price BID", "size BTC", "price ASK"]):
    rng = np.random.default_rng(seed=42)
    new_df = df.copy()
    for col in cols:
        m = rng.random(len(df))
        l1 = 0.008 # NaN
        l2 = 0.009 # Text
        l3 = 0.008 # Negative

        mask1 = m < l1 # NaN
        mask2 = (m >= l1) & (m < l1+l2) # Text
        mask3 = (m >= l1+l2) & (m < l1+l2+l3) # Negative

        new_df.loc[mask1, col] = np.NaN
        new_df.loc[mask2, col] = "?"
        new_df.loc[mask3, col] = -new_df.loc[mask3, col]
    return new_df

In [ ]:
spoiled_df = spoil_df(df)

In [ ]:
spoiled_df.head(60)

As we can see, several question marks, negative values and NaN appeared in the dataframe; those are incorrect data which may hinders our further analysis.

<div>So, how do we identify all that incorrect data and deal with it?</div> 

<b>How to work with incorrect data?</b>

Steps for working with incorrect data:

<ol>
    <li>Identify incorrect data</li>
    <li>Deal with incorrect data</li>
    <li>Correct data format</li>
</ol>


## 1. Identify and handle incorrect values

### Identify incorrect values
### Convert "?" to NaN
In the our dataset, missing data comes with the question mark "?".
We replace "?" with NaN (Not a Number), Python's default missing value marker for reasons of computational speed and convenience. Here we use the function: 
 <pre>.replace(A, B, inplace = True) </pre>
to replace A by B.


In [ ]:
# replace "?" to NaN
spoiled_df.replace("?", np.nan, inplace = True)
spoiled_df.head(60)

### Convert negative values to NaN

To do this, we will perform the following actions:<br>
    1)Let's create a list of column names num_cols that contains the columns "size BID", "price BID", "size BTC", "price ASK" and "price BTC".<br>
    2)Let's convert the column values ​​from the num_cols list to the "float" type.<br>
    3)Let's merge the "ts" column with the list of columns from the num_cols list, applying the .mask() method to replace values ​​less than zero with NaN. Assign the obtained result to the spoiled_df variable.

In [ ]:
# replace negative values to NaN
num_cols = ["size BID", "price BID", "size BTC" ,"price BTC", "size ASK", "price ASK"]
spoiled_df[num_cols] = spoiled_df[num_cols].astype("float")
Saldo = spoiled_df["Saldo"]
spoiled_df = spoiled_df[num_cols].mask(spoiled_df[num_cols] < 0)
spoiled_df["Saldo"] = Saldo

In [ ]:
spoiled_df.head(40)

### Evaluating for Missing Data

The missing values are converted by default. We use the following functions to identify these missing values. There are two methods to detect missing data:

<ol>
    <li><b>.isnull()</b></li>
    <li><b>.notnull()</b></li>
</ol>
The output is a boolean value indicating whether the value that is passed into the argument is in fact missing data.


In [ ]:
missing_data = spoiled_df.isnull()
missing_data.head(60)

"True" means the value is a missing value while "False" means the value is not a missing value.


### Count missing values in each column
<p>
Using a for loop in Python, we can quickly figure out the number of missing values in each column. As mentioned above, "True" represents a missing value and "False" means the value is present in the dataset.  In the body of the for loop the method ".value_counts()" counts the number of "True" values. 
</p>


In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

Based on the summary above, each column has 324 rows of data and three of the columns containing missing data:

<ol>
    <li>"size BID": 8 missing data</li>
    <li>"price BID": 9 missing data</li>
    <li>"size BTC": 11 missing data</li>
    <li>"price ASK": 9 missing data</li>
</ol>


### Deal with missing data
<b>How to deal with missing data?</b>

<ol>
    <li>Drop data<br>
        a. Drop the whole row<br>
        b. Drop the whole column
    </li>
    <li>Replace data<br>
        a. Replace it by mean<br>
        b. Replace it by frequency<br>
        c. Replace it based on other functions
    </li>
</ol>


Whole columns should be dropped only if most entries in the column are empty. In our dataset, none of the columns are empty enough to drop entirely.
We have some freedom in choosing which method to replace data; however, some methods may seem more reasonable than others. Our dataset is tied to time, so we use the pandas interpolator to preserve data quality:

<b>Replace by pandas interpolation:</b>

<ul>
    <li>"size BID": 8 missing data, replace them with frequency
    </li>
    <li>"price BID": 9 missing data, replace them with model output</li>
    <li>"size BTC": 11 missing data, replace them with model output</li>
    <li>"price ASK": 9 missing data, replace them with model output</li>
</ul>




### Replacing with pandas interpolation


Let's try different models for interpolation and take the best one

In [ ]:
def MSE(actual, predicted):
    return np.square(np.subtract(np.array(actual), np.array(predicted))).mean()

def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))
    return mape

In [ ]:
# Setting precision
pd.set_option("display.precision", 10)
# Methods without order
methods = ["linear", "nearest", "slinear", "quadratic", "cubic", "piecewise_polynomial", "pchip", "akima", "cubicspline"]
# Methods with order
order_methods = ["spline", "polynomial"]
performance = pd.DataFrame({"name": [], "MSE": [], "MAPE": []})
for method in methods:
    # print(method)
    nan_rows = spoiled_df["size BTC"].isna()
    interpolated_close = spoiled_df["size BTC"].interpolate(method=method)
    mse = MSE(interpolated_close[nan_rows], df.loc[nan_rows, "size BTC"])
    mape = MAPE(df.loc[nan_rows, "size BTC"], interpolated_close[nan_rows])
    performance.loc[len(performance.index)] = [method, mse, mape]
for method in order_methods:
    for order in [3, 5]:
        # print(f"method: {method}, order: {order}")
        nan_rows = spoiled_df["size BTC"].isna()
        interpolated_close = spoiled_df["size BTC"].interpolate(method=method, order=order)
        mse = MSE(df.loc[nan_rows, "size BTC"], interpolated_close[nan_rows])
        mape = MAPE(df.loc[nan_rows, "size BTC"], interpolated_close[nan_rows])
        performance.loc[len(performance.index)] = [f"{method}_{order}", mse, mape]
        
performance = performance.sort_values(by=["MAPE", "MSE"], ascending=True)
performance["MAPE"] = performance["MAPE"] * 100
performance["MAPE"] = performance["MAPE"].astype("str") 
performance["MAPE"] = performance["MAPE"].str.slice(stop=8) + "%"
performance.head(15)

Since our time inputs are at large intervals our percentages are quite large. If you have a small time interval, you will have lower percentages.

How we can see the best model for interpolation is linear so we'll use it to replace NaN values

In [ ]:
spoiled_df['size BTC'].interpolate(method = 'linear', inplace=True)
spoiled_df.head(40)

In [ ]:
spoiled_df['size BID'].interpolate(method = 'linear', inplace=True)
spoiled_df.head(40)

In [ ]:
spoiled_df['price BID'].interpolate(method = 'linear', inplace=True)
spoiled_df.head(40)

In [ ]:
spoiled_df['price ASK'].interpolate(method = 'linear', inplace=True)
spoiled_df.head(40)

In [ ]:
spoiled_df.isna().sum()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #1: </h1>

<b>Based on the example above, replace NaN in "price ASK" column with the pandas interpolate.</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 
spoiled_df['price ASK'].interpolate(method = 'linear', inplace=True)
spoiled_df.head(60)

<details><summary>Click here for the solution</summary>

```python
spoiled_df['price ASK'].interpolate(method = 'linear', inplace=True)
spoiled_df.head(60)
```

</details>


<b>Good!</b> Now, we have a dataset with no missing values.


## Correct data format
<b>We are almost there!</b>
<p>The last step in data cleaning is checking and making sure that all data is in the correct format (int, float, text or other).</p>

In Pandas, we use:

<p><b>.dtype()</b> to check the data type</p>
<p><b>.astype()</b> to change the data type</p>


### Let's list the data types for each column


In [ ]:
spoiled_df.dtypes

### Convert data types to proper format


<p>As we can see above, our columns have the correct data type, but if later in the tutorial you need to change the data type, I will show you how to do it. Numerical variables should have type "float" or "int", and variables with timestamps have type "datetime". We have to convert data types into a proper format for each column using the "astype()" method.</p>
<p>Let's look at the data conversion method as an example for you.</p>


In [ ]:
#spoiled_df["size BID"] = spoiled_df["size BID"].astype("float64")
#spoiled_df["price BID"] = spoiled_df["price BID"].astype("float64")

<b>Wonderful!</b>

Now we have finally obtained the cleaned dataset with no missing values with all data in its proper format.


# 2. Data Standardization
<p>
Data is usually collected from different agencies in different formats.
(Data standardization is also a term for a particular type of data normalization where we subtract the mean and divide by the standard deviation.)
</p>

<b>What is standardization?</b>

<p>Standardization is the process of transforming data into a common format, allowing the researcher to make the meaningful comparison.
</p>

<b>Example</b>

<p>Transform BTC TO EUR:</p>
<p>We will need to apply <b>data transformation</b> to transform BTC into EUR.</p>


<p>We can do many mathematical operations directly in Pandas.</p>


In [ ]:
spoiled_df.head()

Let's get the exchange rate and convert

In [ ]:
# Convert mpg to USDT by mathematical operation
res = requests.get("https://api.binance.com/sapi/v1/convert/exchangeInfo?fromAsset=BTC&toAsset=EUR")
if res.status_code != 200:
    rate = 0.95
else:
    res = res.json()
    rate = float(res[0]["toAssetMinAmount"])
    
print(f"The exchange rate is 1 BTC = {rate} EUR")

cols_to_convert = ["price BTC"]
for col in cols_to_convert:
    spoiled_df[f"{col}_EUR"] = spoiled_df[col] * rate

# check your transformed data 
spoiled_df[["price BTC_EUR"]].head()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #2: </h1>

<b>According to the example above, transform "price ASK" (BTC price) to EUR and name the column "price_ASK_EUR".</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 

<details><summary>Click here for the solution</summary>

```python
# Convert mpg to USDT by mathematical operation
res = requests.get("https://api.binance.com/sapi/v1/convert/exchangeInfo?fromAsset=BTC&toAsset=EUR")
if res.status_code != 200:
    rate = 0.04588629
else:
    res = res.json()
    rate = float(res[0]["toAssetMinAmount"])
    
print(f"The exchange rate is 1 BTC = {rate} EUR")

cols_to_convert = ["price ASK"]
for col in cols_to_convert:
    spoiled_df[f"{col}_EUR"] = spoiled_df[col] * rate

# check your transformed data 
spoiled_df[["price ASK_EUR"]].head()
```

</details>


# 3. Data Normalization

<b>Why normalization?</b>

<p>Normalization is the process of transforming values of several variables into a similar range. Typical normalizations include scaling the variable so the variable average is 0, scaling the variable so the variance is 1, or scaling the variable so the variable values range from 0 to 1.
</p>

<b>Example</b>

<p>To demonstrate normalization, let's say we want to scale the columns "size BID", "price BID" and "price ASK".</p>
<p><b>Target:</b> would like to normalize those variables so their value ranges from 0 to 1</p>
<p><b>Approach:</b> replace original value by (original value)/(maximum value), sklearn <b>MinMaxScaler</b></p>


In [ ]:
# replace (original value) by (original value)/(maximum value)
spoiled_df["size BID_norm"] = spoiled_df["size BID"] / spoiled_df["size BID"].max()
scaler = MinMaxScaler()
spoiled_df["price BID_norm"] = scaler.fit_transform(spoiled_df["price BID"].to_numpy().reshape(-1, 1))

In [ ]:
spoiled_df[["size BID_norm", "price BID_norm"]].head()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #3: </h1>

<b>According to the example above, normalize the column "price BID" using sklearn <b>MinMaxScaler</b>.</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 

<details><summary>Click here for the solution</summary>

```python
scaler = MinMaxScaler()
spoiled_df["price BID_norm"] = scaler.fit_transform(spoiled_df["price BID"].to_numpy().reshape(-1, 1))


# show the scaled column
spoiled_df[["price BID_norm"]].head()


```

</details>


Here we can see we've normalized "size BID", "price BID" and "price ASK" in the range of \[0,1].


# 3. Binning and indicator variable
<b>Why binning?</b>
<p>
    Binning is a process of transforming continuous numerical variables into discrete categorical 'bins' for grouped analysis.
</p>

<b>Example: </b>

<p>In our dataset, "size BID" is a real valued variable. What if we want to break it down into 3 bins and see which bin sold the most? Can we rearrange them into three ‘bins' to simplify analysis? </p>

<p>We will use the pandas method 'cut' to segment the 'size BID' column into 3 bins.</p>


## Example of Binning Data In Pandas


Let's plot the histogram of "size BID bins" to see what the distribution of "size BID" looks like.


In [ ]:
plt.hist(spoiled_df["size BID"])

# set x/y labels and plot title
plt.xlabel("size BID")
plt.ylabel("count")
plt.title("size BID bins")

In [ ]:
bins = np.linspace(min(df["size BID"]), max(df["size BID"]), 4)
bins

In [ ]:
spoiled_df["size BID"].describe()

We set group  names:


In [ ]:
group_names = ["Low", "Medium", "High"]

We apply the function <b>cut</b> to determine what each value of `spoiled_df["volume"]` belongs to.


In [ ]:
spoiled_df["size BID-binned"] = pd.cut(spoiled_df["size BID"], bins, labels=group_names, include_lowest=True)
spoiled_df[["size BID", "size BID-binned"]].head(20)

Let's see the "size BID" in each bin:


In [ ]:
spoiled_df["size BID-binned"].value_counts()

Let's plot the distribution of each bin:


In [ ]:
plt.bar(group_names, spoiled_df["size BID-binned"].value_counts().sort_index())

# set x/y labels and plot title
plt.xlabel("size BID")
plt.ylabel("count")
plt.title("size BID bins")

<p>
    As we can see we managed to create 3 classes based on "size BID"
</p>

### Bins Visualization
Normally, a histogram is used to visualize the distribution of bins we created above. 


In [ ]:
# draw historgram of attribute "size BID" with bins = 3
plt.hist(spoiled_df["size BID"], bins = 3)

# set x/y labels and plot title
plt.xlabel("size BID")
plt.ylabel("count")
plt.title("size BID bins")

The plot above shows the binning result for the attribute "size BID".


### Indicator Variable (or Dummy Variable)
<b>What is an indicator variable?</b>
<p>
    An indicator variable (or dummy variable) is a numerical variable used to label categories. They are called "dummies" because the numbers themselves don't have inherent meaning. 
</p>

<b>Why we use indicator variables?</b>

<p>
    We use indicator variables so we can use categorical variables for regression analysis in the later modules.
</p>
<b>Example</b>
<p>
    We see the column "size BID-binned" has three unique values: "Low", "Medium" or "High". Regression doesn't understand words, only numbers. To use this attribute in regression analysis, we convert "rec_count-binned" to indicator variables.
</p>

<p>
    We will use pandas' method <b>get_dummies</b> to assign numerical values to different categories of "close". 
</p>


Get the indicator variables and assign it to data frame "dummy_variable\_1":


In [ ]:
dummy_variable_1 = pd.get_dummies(spoiled_df["size BID-binned"], prefix="size BID")
dummy_variable_1.head()

In [ ]:
# merge data frame "spoiled_df" and "dummy_variable_1" 
spoiled_df = pd.concat([spoiled_df, dummy_variable_1], axis=1)

# drop original column "rec_count-binned" from "spoiled_df"
spoiled_df.drop("size BID-binned", axis = 1, inplace=True)

In [ ]:
spoiled_df.head(10)

The last three columns are now the indicator variable representation of the volume variable. They're all 0s and 1s now.


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4: </h1>

<b>Similar to before, create an indicator variable for the column "ts_BTC" (create spoiled_df["ts_BTC_month"] = spoiled_df["ts_BTC"].dt.month as category and then get_dummies from that column and then concatenate the spoiled_df) and merge the dataframes</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


<details><summary>Click here for the solution</summary>

```python
# Create a new columnb
spoiled_df["ts_BTC_month"] = spoiled_df.index.month
# get indicator variables of aspiration and assign it to data frame "dummy_variable_2"
dummy_variable_2 = pd.get_dummies(spoiled_df["ts_BTC_month"], prefix="ts")
# Concatenate df's
spoiled_df = pd.concat([spoiled_df, dummy_variable_2], axis=1)
# Drop original column "ts_month" from "df"
spoiled_df.drop("ts_BTC_month", axis=1, inplace=True)
spoiled_df.head()

```

</details>


# Resampling

<b>What is resampling?</b>
<p>
    Data resampling is any process whereby data is gathered and expressed in a summary form
</p>

After resampling we can use different aggregation functions such as:

<ul>
    <li><b>mean()</b></li>
    <li><b>sum()</b></li>
    <li><b>prod()</b></li>
    <li><b>first()</b></li>
    <li><b>last()</b></li>
    <li><b>min()</b></li>
    <li><b>max()</b></li>
</ul>

In [ ]:
spoiled_df[["price ASK", "size ASK","price BID","size BID","price BTC","size BTC","Saldo"]].resample("1H").agg({"price ASK": "mean","size ASK": "sum","price BID": "mean","size BID": "sum","price BTC":"mean","size BTC":"sum", "Saldo":"mean"})

How we can see we resampled data with aggregation time 1 hour so we reduced amount of the data and generalized it. And we can make assumptions on much wider time window

 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #5: </h1>

<b>Apply resampling to "spoiled_df" with aggregation time 2 hour. Then apply <b>max()</b> function</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


<details><summary>Click here for the solution</summary>

```python
spoiled_df[["price ASK", "size ASK","price BID","size BID","price BTC","size BTC","Saldo","price BTC_EUR","price BID_norm","size BID_norm","size BID_Low","size BID_Medium","size BID_High"]].resample('2h').max().head()
```

</details>


In [ ]:
spoiled_df.to_csv("clean_df.csv", index=True)

Save the new csv:

> Note : The  csv file cannot be viewed in the jupyterlite based SN labs environment.However you can Click <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Module%202/DA0101EN-2-Review-Data-Wrangling.ipynb?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2022-01-01">HERE ???</a> to download the lab notebook (.ipynb) to your local machine and view the csv file once the notebook is executed.


### Thank you for completing this lab!

## Author

<a href="https://author.skills.network/instructors/danyil_zhupnyk">Danyil Zhupnyk</a><br>
<a href="https://author.skills.network/instructors/yaroslav_vyklyuk_2">Prof. Yaroslav Vyklyuk, DrSc, PhD</a><br>
<a href="https://author.skills.network/instructors/mariya_fleychuk">Prof. Mariya Fleychuk, DrSc, PhD</a><br>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2023-03-04        | 1.0     | D.Zhupnyk  | Created Lab                        |

<hr>

## <h3 align="center"> © IBM Corporation 2023. All rights reserved. <h3/>